In [1]:
print('ok')

ok


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
import pypdf

In [13]:
def load_pdf(data):
    loader= DirectoryLoader(data,loader_cls=PyPDFLoader,glob='*.pdf')
    
    documents=loader.load()
    return documents

In [15]:
extracted_data=load_pdf('data/')